# LSTM for March 2021 Report

https://www.altumintelligence.com/articles/a/Time-Series-Prediction-Using-LSTM-Deep-Neural-Networks

https://github.com/jaungiers/LSTM-Neural-Network-for-Time-Series-Prediction

Dataset:

https://data.mendeley.com/datasets/wr3zm9d7n9/1

https://data.mendeley.com/datasets/f4fcrh4tn9/1

Denormalize:

https://gist.github.com/optman/804c810855dfb388668cbbdb09e6199f
    

In [11]:
%%script C:\Users\marci\AppData\Local\Programs\Python\Python35\python.exe
import sys

# C:\Users\marci\AppData\Local\Programs\Python\Python35>python -m pip install keras==2.2.2
#numpy==1.15.0
#pandas==0.23.3
#tensorflow-gpu==1.10.0
#keras==2.2.2
#matplotlib==2.2.2
#tensorflow==1.10.0
#python -m pip install ipython==3.2.1
#python -m pip install ipykernel
#python -m pip install pandas
#python -m pip install ipympl
#python -m pip install sklearn


import matplotlib.pyplot as plt
#%matplotlib inlinen # Does not work b/c conflict with %%script

import numpy as np

import os
import json
import time
import math
from core.data_processor import DataLoader
from core.model import Model
from sklearn.preprocessing import MinMaxScaler


__author__ = "Marcia Hon revised from Jakob Aungiers"
__copyright__ = "Marcia Hon 2021 revised from Jakob Aungiers 2018"
__version__ = "1.0.0 revised from 2.0.0"
__license__ = "Ryerson revised from MIT"


def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white', figsize=[30, 20])
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    ax.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.savefig('plot_results.png')
    #plt.show()


def plot_results_multiple(predicted_data, true_data, prediction_len):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
	# Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_len)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()


def main():
    configs = json.load(open('config.json', 'r'))
    if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

    data = DataLoader(
        os.path.join('data', configs['data']['filename']),
        configs['data']['train_test_split'],
        configs['data']['columns']
    )

    model = Model()
    model.build_model(configs)
    x, y = data.get_train_data(
        seq_len=configs['data']['sequence_length'],
        normalise=configs['data']['normalise']
    )

    '''
	# in-memory training
	model.train(
		x,
		y,
		epochs = configs['training']['epochs'],
		batch_size = configs['training']['batch_size'],
		save_dir = configs['model']['save_dir']
	)
	'''
    # out-of memory generative training
    steps_per_epoch = math.ceil((data.len_train - configs['data']['sequence_length']) / configs['training']['batch_size'])
    model.train_generator(
        data_gen=data.generate_train_batch(
            seq_len=configs['data']['sequence_length'],
            batch_size=configs['training']['batch_size'],
            normalise=configs['data']['normalise']
        ),
        epochs=configs['training']['epochs'],
        batch_size=configs['training']['batch_size'],
        steps_per_epoch=steps_per_epoch,
        save_dir=configs['model']['save_dir']
    )

    x_test, y_test = data.get_test_data(
        seq_len=configs['data']['sequence_length'],
        normalise=configs['data']['normalise']
    )

    #predictions = model.predict_sequences_multiple(x_test, configs['data']['sequence_length'], configs['data']['sequence_length'])
    #predictions = model.predict_sequence_full(x_test, configs['data']['sequence_length'])

    #Denormalize - Point by Point : use with normalize set to true in config.json
    #Since it is point-by-point with no windows, we can do regular normalization
    #Normalization Function: ($normalized * ($max - $min) + $min);
    predictions = model.predict_point_by_point(x_test)
    dn_predictions = (np.array(predictions) * (data.base_max - data.base_min)) + data.base_min
    dn_ytest = (y_test * (data.base_max - data.base_min)) + data.base_min
    plot_results(dn_predictions, dn_ytest)
  

if __name__ == '__main__':
    main()

[Model] Model Compiled
Time taken: 0:00:00.489974
[Model] Training Started
[Model] 2 epochs, 32 batch size, 835 batches per epoch
Epoch 1/2
835/835 [==============================] - 65s 78ms/step - loss: 0.0061
Epoch 2/2
835/835 [==============================] - 65s 78ms/step - loss: 0.0014
[Model] Training Completed. Model saved as saved_models\07022021-092828-e2.h5
Time taken: 0:02:11.862086
[Model] Predicting Point-by-Point...


2021-02-07 09:28:24.649784: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_100.dll'; dlerror: cudart64_100.dll not found
2021-02-07 09:28:24.649811: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Using TensorFlow backend.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





2021-02-07 09:28:30.403093: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2021-02-07 09:28:30.423680: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.8475
pciBusID: 0000:01:00.0
2021-02-07 09:28:30.424224: W tensorflow/stream_executor/platform/de